# Coinbase Pro API Test

The sandbox version of the coinbase API is at:

   https://public.sandbox.pro.coinbase.com
    
Cbpro Python library repository:

   https://github.com/danpaquin/coinbasepro-python
    
API url (sandbox):

   https://api-public.sandbox.pro.coinbase.com

In [15]:
%config Completer.use_jedi = False       #without this line, autocomplete seemed to not be working, so I am putting it here. source: https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working

In [16]:
import pandas as pd
import cbpro
from auth_credentials import (api_secret,
                              api_key, 
                              api_pass)

## WebsocketClient Class

Here we inherit from the WebsocketClient class and overload methods `on_open()`, `on_message()`, and `on_close()`

In [17]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url           = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val   = msg.get('time',('-'*27))
            price_val  = msg.get('price',None)
            if price_val is not None:
                price_val = float(price_val)
            product_id = msg.get('product_id',None)
            
            print(f"{time_val:30} \
                {price_val:.3f} \
                {product_id}\tchannel type:{msg_type}")
    
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [13]:
stream = TextWebsocketClient(products=['BTC-USD'],channels=['ticker'])
stream.start()

2021-01-24T05:02:50.325720Z                    32766.430                 BTC-USD	channel type:ticker
2021-01-24T05:02:55.317811Z                    32766.410                 BTC-USD	channel type:ticker
2021-01-24T05:02:56.227170Z                    32766.420                 BTC-USD	channel type:ticker
2021-01-24T05:02:56.551063Z                    32766.420                 BTC-USD	channel type:ticker
2021-01-24T05:02:56.868469Z                    32766.420                 BTC-USD	channel type:ticker
2021-01-24T05:02:57.193821Z                    32766.420                 BTC-USD	channel type:ticker
2021-01-24T05:02:57.521818Z                    32766.420                 BTC-USD	channel type:ticker
2021-01-24T05:02:59.353735Z                    32766.420                 BTC-USD	channel type:ticker
2021-01-24T05:03:00.318903Z                    32766.420                 BTC-USD	channel type:ticker


In [10]:
print("Now you see me")

Now you see me
2021-01-24T04:55:34.915892Z    32691.230 BTC-USD	channel type:ticker
2021-01-24T04:55:35.733581Z    32691.230 BTC-USD	channel type:ticker
2021-01-24T04:55:40.321576Z    32691.240 BTC-USD	channel type:ticker
2021-01-24T04:55:44.549054Z    32707.520 BTC-USD	channel type:ticker
2021-01-24T04:55:44.549054Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:45.317080Z    32707.510 BTC-USD	channel type:ticker
2021-01-24T04:55:46.003897Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:46.328381Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:46.648948Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:46.985645Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:47.305601Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:50.332261Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:50.503458Z    32707.530 BTC-USD	channel type:ticker
2021-01-24T04:55:55.324106Z    32707.520 BTC-USD	channel type:ticker
2021-01-24T04:55:55

In [14]:
stream.close()

2021-01-24T05:03:05.329153Z                    32766.410                 BTC-USD	channel type:ticker
2021-01-24T05:03:06.191011Z                    32766.410                 BTC-USD	channel type:ticker
<---Websocket connection closed--->
	Total messages: 12


## AuthenticatedClient Class


An authenticated client allows us to access private account information and can be done with the class `AuthenticatedClient` to handle this type of connection.

In [20]:
url='https://api-public.sandbox.pro.coinbase.com'

client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)

## Placing a Market Order

Here we see how to place market orders through cbpro with an authenticated client object.

In [ ]:
client.place_market_order(product_id='BTC-USD',side='buy',size=1)

## Check Account History

First we need to know the id of the account we want to check. Therefore, we use `client.get_accounts()` to obtain the id of the target account and we then check its history with `client.get_account_history(<account id>)`.

In [32]:
accounts = client.get_accounts()

In [33]:
for acc in accounts:
    currency = acc.get('currency')
    if currency=='BTC':
        acc_id = acc.get('id')

In [34]:
acc_history = client.get_account_history(acc_id)

In [ ]:
import json

for hist in acc_history:
    print(json.dumps(hist,indent=1))